In [4]:
%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt
import gymnasium as gym
import time


from gym.envs.registration import register

theId = 'FrozenLake-v1'

#try: 
    register(
        id= theId,
        entry_point='gym.envs.toy_text:FrozenLakeEnv',
        kwargs={'map_name' : '4x4', 'is_slippery': False},
        max_episode_steps=100,
        reward_threshold=0.78, # optimum = .8196
    )
except: 
    print('already registered')

In [5]:

theId = 'FrozenLake-v1'
env = gym.make(theId, desc=None, map_name="4x4", is_slippery=True, render_mode="human")
env.reset()


for step in range(15):
    env.render()
    action = env.action_space.sample()
    observation, reward, terminated, truncated, info = env.step(action)
    env.step
    time.sleep(0.3)
    
    if termindated or truncated:
        env.reset()
    
env.close()

NameError: name 'termindated' is not defined